In [1]:
# 此篇要做 2 個以上的群中心
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import MiniBatchKMeans
import random
from sklearn.neighbors import NearestNeighbors
import numpy as np
import collections
from collections import Counter

import os;
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from matplotlib import pyplot
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from imblearn.over_sampling import ADASYN
from sklearn import cluster, datasets, metrics
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
import statistics

In [2]:
def find_maj(sample_class):
    counter = Counter(sample_class);
    maj = list(dict(counter.most_common(1)).keys())
    maj = "".join(maj)
    return  maj


def classprocess(output):
    c = Counter(output)
    datagap = []
    maj = find_maj(output)
    maj_num = dict(c)[find_maj(output)]
    for className, number in c.items(): 
        #print(className," ",number)
        print(number)
        temp = np.array([className,(maj_num - number)])
        datagap.append(temp)
    return datagap

In [3]:
# 由於 RCSMOTE 出現問題 所以改用 boderline 加 SMOTE 加 borderline 加 ADASYN
import os
path = "/Users/emily/Desktop/Research/oversampling_python/data/"
folderName = 'haberman-5-fold' # yeast6-5-fold'#'haberman-5-fold' #'abalone19-5-fold' #'abalone19-5-fold' # 
os.chdir(path+ folderName)
dirs = os.listdir(path+ folderName)
train = []
test = []

for i in dirs:
    if("xlsx" in i):
        if("tra" in i):
            train.append(i)
        elif("tst" in i):
            test.append(i)

In [4]:
train

['haberman-5-2tra.xlsx',
 'haberman-5-3tra.xlsx',
 'haberman-5-1tra.xlsx',
 'haberman-5-5tra.xlsx',
 'haberman-5-4tra.xlsx']

SMOTE 兩個群中心

In [ ]:
data.shape[0]

In [6]:
# SMOTE 群中心 index
# 兩個 center
from imblearn.over_sampling import  SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []
centerSMOTE1 = []
centerSMOTE2 = []
countfor = 0;
centerSMOTEall=[]
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverSMOTE = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    
    overSMOTE.append(alloverSMOTE)
    # 整理程式碼 參數使用可填入 多少個 cluster 可用 for 迴圈
    for i in range(len(classCount)):
        
        countfor = math.floor(int(classCount[i][1])*0.5); # 全部需要的數量 無條件捨去
        
        countfor_center1 = math.ceil(countfor*0.3); # 第一個群中心的數量  佔全部的多少
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        countfor_center2 = int(countfor*0.7); # 第二個群中心的數量 
        if(countfor>0):
            kmeans = KMeans(n_clusters=2)
            dtemp = pd.DataFrame(overSMOTE[ii])
            #dd = dtemp.iloc[originlen:,:]
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            #print(dd)
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            #print(X)
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for centerindex in range(len(centers)): 
                for i in range(X.shape[0]-1): # 列
                    distance = []
                    temp = 0;
                    for j in range(X.shape[1]-1):#9 行
                        temp = pow((centers[centerindex][j]-X.iloc[i][j]),2)  
                        tempindata[i] = temp
               
                
                distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
                #distanceindex.append(distancesortemp[ii][0]+originlen)
                
                
                if(centerindex == 0):
                    #print(distancesortemp[:countfor_center1])
                    #distancesortemp = distancesortemp[:countfor_center1]
                    centerSMOTE1.append(distancesortemp[:countfor_center1])
                elif(centerindex==1):
                    centerSMOTE2.append(distancesortemp[:countfor_center2])
        
    
    tempcenter = centerSMOTE1[ii] + centerSMOTE2[ii]

    centerSMOTEall.append(tempcenter)
    



180
65
180
65
180
64
180
65
180
65


In [7]:
# centerSMOTEmin 只是 index ，value 是取出值
centerSMOTEvalue =[]
for tr in train:
    data = pd.read_excel(tr,index_col=0)
    originlen = len(data)
    for i in range(len(centerSMOTEall)):
        alltemp = []
        for j in range(len(centerSMOTEall[i])):
            indexSMOTE = centerSMOTEall[i][j][0]+originlen
            #tempSMOTE = list(overSMOTE[i][indexSMOTE])
            alltemp.append(list(overSMOTE[i].iloc[indexSMOTE]))
        centerSMOTEvalue.append(alltemp)

In [8]:
# ADASYN 群中心 兩個 center index

from imblearn.over_sampling import  ADASYN
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverADASYN = []
overADASYN = []
#randomADASYN = []
centerADASYN1 = []
centerADASYN2 = []
countfor = 0;
centerADASYNall=[]
for ii,i in enumerate(train):
    #randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    
   
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverADASYN = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    
    overADASYN.append(alloverADASYN)
    
    # 整理程式碼 參數使用可填入 多少個 cluster 可用 for 迴圈
    for i in range(len(classCount)):
        
        countfor = math.floor(int(classCount[i][1])*0.5); # 全部需要的數量 無條件捨去
        
        countfor_center1 = math.ceil(countfor*0.3); # 第一個群中心的數量  佔全部的多少
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        countfor_center2 = int(countfor*0.7); # 第二個群中心的數量 
        if(countfor>0):
            kmeans = KMeans(n_clusters=2)
            dtemp = pd.DataFrame(overADASYN[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for centerindex in range(len(centers)): 
                for i in range(X.shape[0]-1): # 列
                    distance = []
                    temp = 0;
                    for j in range(X.shape[1]-1):#9 行
                        temp = pow((centers[centerindex][j]-X.iloc[i][j]),2)  
                        tempindata[i] = temp
            
                distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
                if(centerindex == 0):
                    #print(distancesortemp[:countfor_center1])
                    centerADASYN1.append(distancesortemp[:countfor_center1])
                elif(centerindex==1):
                    centerADASYN2.append(distancesortemp[:countfor_center2])
        

    tempcenter = centerADASYN1[ii] + centerADASYN2[ii]

    centerADASYNall.append(tempcenter)
    



180
65
180
65
180
64
180
65
180
65


In [9]:
# centerADASYN 只是 index ，value 是取出值
centerADASYNvalue =[]
for tr in train:
    data = pd.read_excel(tr,index_col=0)
    originlen = len(data)
    for i in range(len(centerADASYNall)):
        alltemp = []
        for j in range(len(centerADASYNall[i])):
            indexADASYN = centerADASYNall[i][j][0] + originlen
            #tempSMOTE = list(overSMOTE[i][indexSMOTE])
            alltemp.append(list(overADASYN[i].iloc[indexADASYN]))
        centerADASYNvalue.append(alltemp)

In [10]:
# 合併 小類 center ADASYN + SMOTE 各 
allCenter = []
temp = []
for i in range(len(centerADASYNvalue)):
    temp = centerADASYNvalue[i] + centerSMOTEvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenter.append(temp)

len(allCenter[1])

114

In [11]:
# # 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

mergeRandom = []
accuracies=[]
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
 
    #data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T

    #allCenterminHalf[index] = pd.DataFrame(allCenterminHalf[index],columns=data.columns)
    mergeRandom = pd.concat([data,allCenter[index]],axis=0)

    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
   
    le = preprocessing.LabelEncoder()
    
 

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


0.8011111111111111


Nothing 

In [ ]:
# 不做任何處理
from sklearn import preprocessing
from imblearn.over_sampling import ADASYN
from sklearn import cluster, datasets, metrics
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
import statistics
from sklearn.metrics import roc_auc_score
accuracies=[]
for ii,i in enumerate(train):
    
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
   
        
    tempover = []
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata,output)
    # 把 X_smote 跟 y_smote 和在一起
    
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1]
    counter = Counter(test_y)
    """
    for k,v in counter.items():
	    per = v / len(output) * 100
	    print("class",k,"數量：",v,"percentage",'%.3f' %per,"%")
	
    pyplot.bar(counter.keys(), counter.values())   
    pyplot.show()  
    """
    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


ADASYN 

In [ ]:
# ADASYN
from sklearn import preprocessing
from imblearn.over_sampling import ADASYN
from sklearn import cluster, datasets, metrics
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
import statistics
accuracies=[]
for ii,i in enumerate(train):
    
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,data.shape[1] -1];
    finaldata = data.iloc[:,:data.shape[1] -1]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    adsn = ADASYN()
    new_X, new_y = adsn.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(new_X, new_y)
    #clf = clf.fit(finaldata,output)
    newDataCount = len(new_X) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


SMOTE 0.8220151828847481

In [ ]:
# SMOTE 
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn import cluster, datasets, metrics
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from matplotlib import pyplot
import statistics
accuracies=[]
for ii,i in enumerate(train):
    
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    smote = SMOTE()
    new_X, new_y = smote.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y
    print(len(new_X),"dasd")
    clf = DecisionTreeClassifier()
    clf = clf.fit(new_X, new_y)
    newDataCount = len(new_X) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    counter = Counter(test_y)
    
    counter = Counter(output)
 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    
    
mean = statistics.mean(accuracies)
print(mean)

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


Borderline-1  0.7979468599033817

In [ ]:
# Borderline-1
from sklearn import preprocessing
from imblearn.over_sampling import BorderlineSMOTE
from sklearn import cluster, datasets, metrics
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from matplotlib import pyplot
import statistics
accuracies=[]
for ii,i in enumerate(train):
    
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    adsn =  BorderlineSMOTE(random_state=42,kind="borderline-1")
    new_X, new_y = adsn.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(new_X, new_y)
    newDataCount = len(new_X) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    counter = Counter(test_y)
    
    counter = Counter(output)
    """
    for k,v in counter.items():
	    per = v / len(output) * 100
	    print("class",k,"數量：",v,"percentage",'%.3f' %per,"%")
	
    pyplot.bar(counter.keys(), counter.values())
    pyplot.show()
    """

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    #accuracy = metrics.accuracy_score(test_y, test_y_predicted)
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    
    
mean = statistics.mean(accuracies)
print(mean)

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


Borderline-2 0.8260006901311249

In [ ]:
# Borderline-2
from sklearn import preprocessing
from imblearn.over_sampling import BorderlineSMOTE
from sklearn import cluster, datasets, metrics
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from matplotlib import pyplot
import statistics
accuracies=[]
for ii,i in enumerate(train):
    
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    Border2 =  BorderlineSMOTE(random_state=42,kind="borderline-2")
    new_X, new_y = Border2.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(new_X, new_y)
    newDataCount = len(new_X) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    counter = Counter(test_y)
    
    counter = Counter(output)
    

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    # 精確度
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    
    
mean = statistics.mean(accuracies)
print(mean)

#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
def find_maj(sample_class):
    counter = Counter(sample_class);
    maj = list(dict(counter.most_common(1)).keys())
    maj = "".join(maj)
    return  maj


def classprocess(output):
    c = Counter(output)
    datagap = []
    maj = find_maj(output)
    maj_num = dict(c)[find_maj(output)]
    for className, number in c.items(): 
        #print(className," ",number)
        print(number)
        temp = np.array([className,(maj_num - number)])
        datagap.append(temp)
    return datagap

In [ ]:
# random SMOTE 取 7 成
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    output = data.iloc[:,l];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    print(len(X_smote),"x_smote")
    # 把 X_smote 跟 y_smote 和在一起
    """
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    """
    alloverSMOTE = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    overSMOTE.append(alloverSMOTE)
    for i in range(len(classCount)):
        #print(classCount[i],"ffksdl;")
        count = math.floor(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []
    #print(overSMOTE)
    
    
    for index in randomIndex:
        randomtemp.append(overSMOTE[ii].iloc[index,:])
    #print(randomtemp)
   
    
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    #print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
     
np.array(randomSMOTE).shape


#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# random ADsyn 取 3 成
from imblearn.over_sampling import ADASYN
alloverAdsyn = []
overAdsyn = []
randomAdsyn = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = data.iloc[:,l];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    print(len(X_smote),"x_smote")
    # 把 X_smote 跟 y_smote 和在一起
   
    alloverAdsyn = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    overAdsyn.append(alloverAdsyn)
    for i in range(len(classCount)):
        #print(classCount[i],"ffksdl;")
        count = math.ceil(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []
    #print(overSMOTE)
    
    
    for index in randomIndex:
        randomtemp.append(overAdsyn[ii].iloc[index,:])
    #print(randomtemp)
   
    
    randomAdsyn.append(randomtemp)
    print(ii," ",len(randomtemp))
    #print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
     
np.array(randomAdsyn).shape


#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
# random borderline2
from imblearn.over_sampling import BorderlineSMOTE
alloverBorder2 = []
overBorder2 = []
randomBorder2 = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = data.iloc[:,l];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    over = BorderlineSMOTE(random_state=42,kind="borderline-2")
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    print(len(X_smote),"x_smote")
    # 把 X_smote 跟 y_smote 和在一起
   
    alloverBorder2 = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    overBorder2.append(alloverBorder2)
    for i in range(len(classCount)):
        #print(classCount[i],"ffksdl;")
        if(classCount[i][1] > 0):
            count = math.ceil(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
            randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []
    #print(overSMOTE)
    
    
    for index in randomIndex:
        randomtemp.append(overBorder2[ii].iloc[index,:])
    #print(randomtemp)
   
    
    randomBorder2.append(randomtemp)
    print(ii," ",len(randomtemp))
    #print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
     
np.array(randomBorder2).shape


#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
allRandomHalf = []
temp = []
# 合併 Adsyn SMOTE
for i in range(len(randomAdsyn)):
    #temp = randomAdsyn[i] + randomSMOTE[i]
    randomAdsynpd = pd.DataFrame(randomAdsyn[i])
    randomSMOTEpd = pd.DataFrame(randomSMOTE[i])
    RandomHalf = pd.concat([randomAdsynpd,randomSMOTEpd],axis=0)
    allRandomHalf.append(RandomHalf)

In [ ]:
allRandomHalf = []
temp = []
# 合併 Adsyn Border2
for i in range(len(randomAdsyn)):
    #temp = randomAdsyn[i] + randomSMOTE[i]
    randomAdsynpd = pd.DataFrame(randomAdsyn[i])
    randomBorder2pd = pd.DataFrame(randomBorder2[i])
    RandomHalf = pd.concat([randomAdsynpd,randomBorder2pd],axis=0)
    allRandomHalf.append(RandomHalf)

In [ ]:
# random 合併三個
allRandomHalf = []
temp = []

for i in range(len(randomAdsyn)):
    #temp = randomAdsyn[i] + randomSMOTE[i]
    randomAdsynpd = pd.DataFrame(randomAdsyn[i])
    randomSMOTEpd = pd.DataFrame(randomSMOTE[i])
    randomBorder2pd = pd.DataFrame(randomBorder2[i])
    RandomHalf = pd.concat([randomAdsynpd,randomSMOTEpd],axis=0)
    RandomHalf = pd.concat([RandomHalf,randomBorder2pd],axis=0)
    allRandomHalf.append(RandomHalf)

In [ ]:
# 跟原始資料合併
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T
    mergeRandom = pd.concat([data,allRandomHalf[index]],axis=0)
    #print(mergeRandom)
    
    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


In [ ]:
counter = Counter(output)
counter

In [ ]:
# 跟原始資料合併
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T
    mergeRandom = pd.concat([data,allRandomHalf[index]],axis=0)
    #print(mergeRandom)
    
    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])

    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
    test_y = le.fit_transform(test_y)
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


In [ ]:
len(allRandomHalf[0])

SMOTE 0.7  ADASYN 0.3  準確度 0.7989130
SMOTE 0.3  ADASYN 0.7  準確度 0.7989130


群中心 for 個別

In [ ]:
from sklearn.cluster import KMeans  
kmeans = KMeans(n_clusters=1)
X = dtemp.iloc[:,:dtemp.shape[1]-1]
kmeans.fit(X)

In [ ]:
data.shape[1]
output = data.iloc[:,data.shape[1]-1];
output[:]

SMOTE 群中心

In [ ]:
# SMOTE 群中心 index
from imblearn.over_sampling import  SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []
centerSMOTE = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverSMOTE = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    
    overSMOTE.append(alloverSMOTE)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.3); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overSMOTE[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerSMOTE.append(distancesortemp[:countfor])

    

print(len(centerSMOTE[0]))

In [ ]:
# centerSMOTEmin 只是 index ，value 是取出值
centerSMOTEvalue =[]
for i in range(len(centerSMOTE)):
    alltemp = []
    for j in range(len(centerSMOTE[i])):
        indexSMOTE = centerSMOTE[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(overSMOTE[i].iloc[indexSMOTE]))
    centerSMOTEvalue.append(alltemp)

In [ ]:
len(centerSMOTEvalue[0])

ADASYN 群中心

In [ ]:
# ADASYN 群中心 index
from imblearn.over_sampling import  ADASYN
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverADASYN = []
overADASYN = []
randomADASYN = []
centerADASYN = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverADASYN = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    
    overADASYN.append(alloverADASYN)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.7); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overADASYN[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerADASYN.append(distancesortemp[:countfor])

    

print(len(centerADASYN[0]))

In [ ]:
# centerADASYN 只是 index ，value 是取出值
centerADASYNvalue =[]
for i in range(len(centerADASYN)):
    alltemp = []
    for j in range(len(centerADASYN[i])):
        indexSMOTE = centerADASYN[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(overADASYN[i].iloc[indexSMOTE]))
    centerADASYNvalue.append(alltemp)

In [ ]:
# 合併 小類 center ADASYN + SMOTE 各 
allCenterHalf = []
temp = []
for i in range(len(centerADASYN)):
    temp = centerADASYNvalue[i] + centerSMOTEvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterHalf.append(temp)

len(allCenterHalf[1])

In [ ]:
# # 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T

    #allCenterminHalf[index] = pd.DataFrame(allCenterminHalf[index],columns=data.columns)
    mergeRandom = pd.concat([data,allCenterHalf[index]],axis=0)

    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
    #test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])
    le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


計算原資料小類10筆的群中心, 分別選出離此小類群中心各一定比例的資料

In [ ]:
# collect 小類 以及算出 群中心
from sklearn.cluster import KMeans 
mincenter = []
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    mintype = 0;
    minlist = [] # collect 小類數據集
    for i in range(len(classCount)):
        if(classCount[i][1] != '0'):
            mintype = classCount[i][0];
    for j in range(data.shape[0]):
        if(output.iloc[j] == mintype):
            minlist.append(data.iloc[j,:])
    minpd = pd.DataFrame(minlist)
    #minpd = minpd.reset_index()
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(minpd.iloc[:,:minpd.shape[1]-1])
    #y_kmeans = kmeans.predict(X)
    centers = kmeans.cluster_centers_
    mincenter.extend(centers)
    


In [ ]:
# 抓出距離小類群中心最近的一半 data 
# SMOTE 
from imblearn.over_sampling import  SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []
centerSMOTEmin = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverSMOTE = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    
    overSMOTE.append(alloverSMOTE)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            #kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overSMOTE[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            #kmeans.fit(X)
            #y_kmeans = kmeans.predict(X)
            #centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            #centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
        
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((mincenter[ii][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerSMOTEmin.append(distancesortemp[:countfor])

    

print(len(centerSMOTEmin[0]))

In [ ]:
# centerSMOTEmin 只是 index ，value 是取出值
centerSMOTEvalue =[]
for i in range(len(centerSMOTEmin)):
    alltemp = []
    for j in range(len(centerSMOTEmin[i])):
        indexSMOTE = centerSMOTEmin[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(overSMOTE[i].iloc[indexSMOTE]))
    centerSMOTEvalue.append(alltemp)
  


In [ ]:
# 抓出距離小類群中心最近的一半 data 
# ADASYN
from imblearn.over_sampling import ADASYN
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverADASYN = []
overADASYN = []
randomADASYN = []
centerADASYNmin = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverADASYN = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後全部的數據
    
    overADASYN.append(alloverADASYN)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            #kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overADASYN[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            #kmeans.fit(X)
            #y_kmeans = kmeans.predict(X)
            #centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            #centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
        
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((mincenter[ii][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerADASYNmin.append(distancesortemp[:countfor])

    

print(len(centerADASYNmin[0]))

In [ ]:
centerADASYNvalue =[]
for i in range(len(centerADASYNmin)):
    alltemp = []
    for j in range(len(centerADASYNmin[i])):
        indexSMOTE = centerADASYNmin[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(overADASYN[i].iloc[indexSMOTE]))
    centerADASYNvalue.append(alltemp)

In [ ]:
# 合併 小類 center ADASYN + SMOTE 各 
allCenterminHalf = []
temp = []
for i in range(len(centerADASYNmin)):
    temp = centerADASYNvalue[i] + centerSMOTEvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterminHalf.append(temp)

len(allCenterminHalf[2])

In [ ]:
# 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T

    #allCenterminHalf[index] = pd.DataFrame(allCenterminHalf[index],columns=data.columns)
    mergeRandom = pd.concat([data,allCenterminHalf[index]],axis=0)

    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
    #test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])
    le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


計算原資料小類10筆的群中心, 然後將 OS1 和 OS2 產生的180筆選最接近此群中心的 90 筆

In [ ]:
# random SMOTE 取 7 成
alloverSMOTE = []
overSMOTE = []
randomSMOTE = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    #data.Class= data.Class.str.replace("\n", "").str.strip()
    output = data.iloc[:,l];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:l]
    #finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    over = SMOTE()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    #print(len(X_smote),"x_smote")
    # 把 X_smote 跟 y_smote 和在一起
    """
    for index,element in enumerate(X_smote):
        temp = np.append(element,[y_smote[index]])
        alloverSMOTE.append(temp)
    overSMOTE.append(alloverSMOTE)
    alloverSMOTE =[]
    """
    alloverSMOTE = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    overSMOTE.append(alloverSMOTE)
    for i in range(len(classCount)):
        #print(classCount[i],"ffksdl;")
        count = math.floor(int(classCount[i][1])); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []
    #print(overSMOTE)
    
    
    for index in randomIndex:
        randomtemp.append(overSMOTE[ii].iloc[index,:])
    #print(randomtemp)
   
    
    randomSMOTE.append(randomtemp)
    print(ii," ",len(randomtemp))
    #print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
     
np.array(randomSMOTE).shape


#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
len(overSMOTE[3])

In [ ]:
# random ADsyn 取 3 成
from imblearn.over_sampling import ADASYN
alloverAdsyn = []
overAdsyn = []
randomAdsyn = []

for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    
    data.Class= data.Class.str.replace("\n", "").str.strip()
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    
    print(len(X_smote),"x_smote")
    # 把 X_smote 跟 y_smote 和在一起
   
    alloverAdsyn = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後的數據
    #print(len(alloverADASYN),"length")
    overAdsyn.append(alloverAdsyn)

    
    for i in range(len(classCount)):
        #print(classCount[i],"ffksdl;")
        count = math.ceil(int(classCount[i][1])); # 要產生多少數據  無條件捨去
        randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
    
    randomtemp = []
    #print(overSMOTE)
    
    
    for index in randomIndex:
        randomtemp.append(overAdsyn[ii].iloc[index,:])
    #print(randomtemp)
   
    
    randomAdsyn.append(randomtemp)
    #print(ii," ",len(randomtemp))
    #print(np.array(randomSMOTE).shape)
    #print("actual",len(randomSMOTE[ii]))
    
    #print("we",len(randomSMOTE[ii]))
     
np.array(randomAdsyn).shape


#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
len(randomSMOTE[0])

In [ ]:
allRandomHalf = []
temp = []
# 合併 Adsyn SMOTE
for i in range(len(randomAdsyn)):
    #temp = randomAdsyn[i] + randomSMOTE[i]
    randomAdsynpd = pd.DataFrame(randomAdsyn[i])
    randomSMOTEpd = pd.DataFrame(randomSMOTE[i])
    RandomHalf = pd.concat([randomAdsynpd,randomSMOTEpd],axis=0)
    allRandomHalf.append(RandomHalf)

In [ ]:
len(allRandomHalf[3])

In [ ]:
# 抓出距離小類群中心最近的一半 data 
# ADASYN
from imblearn.over_sampling import ADASYN
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import math
from sklearn.cluster import KMeans  
alloverADASYN = []
overADASYN = []
randomADASYN = []
centerADASYNmin = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    #le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    over = ADASYN()
    X_smote,y_smote = over.fit_resample(finaldata,output)
    newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    # 把 X_smote 跟 y_smote 和在一起
    alloverADASYN = pd.concat([X_smote,y_smote],axis=1) # SMOTE 完後全部的數據
    
    overADASYN.append(alloverADASYN)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.5); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            #kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overADASYN[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            #kmeans.fit(X)
            #y_kmeans = kmeans.predict(X)
            #centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            #centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
        
            for i in range(X.shape[0]): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]):#9 行
                    temp = pow((mincenter[ii][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerADASYNmin.append(distancesortemp[:countfor])

    

print(len(centerADASYNmin[0]))

In [ ]:
X.shape[1]

In [ ]:
# collect 小類 以及算出 群中心
from sklearn.cluster import KMeans 
mincenter = []
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    mintype = 0;
    minlist = [] # collect 小類數據集
    for i in range(len(classCount)):
        if(classCount[i][1] != '0'):
            mintype = classCount[i][0];
    for j in range(data.shape[0]):
        if(output.iloc[j] == mintype):
            minlist.append(data.iloc[j,:])
    minpd = pd.DataFrame(minlist)
    #minpd = minpd.reset_index()
    kmeans = KMeans(n_clusters=1)
    kmeans.fit(minpd.iloc[:,:minpd.shape[1]-1])
    #y_kmeans = kmeans.predict(X)
    centers = kmeans.cluster_centers_
    mincenter.extend(centers)
    


In [ ]:
for i in range(4):
    print(i)

In [ ]:
# 整合所有合成 data 後 取出鄰近小類群中心的 data
centerallmin = []

for ii,element in enumerate(train):
    data = pd.read_excel(element,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]
    
    X = allRandomHalf[ii].iloc[:,:allRandomHalf[ii].shape[1]-1]
    #print(X)
    for i in range(len(classCount)):
            countfor = math.floor(int(classCount[i][1])); # 要產生多少數據  無條件捨去
            #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
            print("classcount",countfor)

            if(countfor>0):
                distance = []
                X = X.astype('float64')
                #centers = centers.astype('float64')
                tempindata = {}
                distancesortemp = []
                for ro in range(X.shape[0]):
                    for j in range(X.shape[1]):#9 行
                        temp = pow((mincenter[ii][j]-X.iloc[ro][j]),2)  
                       
                        tempindata[ro] = temp
            
                distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
                #print(distancesortemp)
     
                centerallmin.append(distancesortemp[:countfor])

    

print(len(centerallmin[0]))

In [ ]:
centerallvalue =[]
for i in range(len(centerallmin)):
    alltemp = []
    for j in range(len(centerallmin[i])):
        indexSMOTE = centerallmin[i][j][0]
        #tempSMOTE = list(overSMOTE[i][indexSMOTE])
        alltemp.append(list(allRandomHalf[i].iloc[indexSMOTE]))
    
    centerallvalue.append(alltemp)


In [ ]:
data.columns

In [ ]:
# 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    #data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T

    #allCenterminHalf[index] = pd.DataFrame(allCenterminHalf[index],columns=data.columns)
    centerallvaluepd = pd.DataFrame(centerallvalue[index],columns=data.columns)
    mergeRandom = pd.concat([data,centerallvaluepd],axis=0)

    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
    #test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])
    le = preprocessing.LabelEncoder()
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)




分別產生各 data

In [ ]:
# SMOTE 利用 sampling strategy 產出不同資料數
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn import cluster, datasets, metrics
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from matplotlib import pyplot
import statistics
sampleSMOTE = []
for ii,i in enumerate(train):
    
    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    #finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    tempover = []
    smote = SMOTE(sampling_strategy=0.5)
    originlen = len(data) #要去掉原先的 data
    new_X, new_y = smote.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y

    temp = pd.concat([new_X,new_y],axis=1)
    temp = temp.iloc[originlen:,:]
    sampleSMOTE.append(temp)
   
#現在 randomSMOTE 存的是 random 的 SMOTE 生成 data


In [ ]:
#  ADASYN 利用 sampling strategy 產出
from sklearn import preprocessing
from imblearn.over_sampling import ADASYN
from sklearn import cluster, datasets, metrics
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from numpy import mean
from matplotlib import pyplot
import statistics
sampleADASYN = []
for ii,i in enumerate(train):

    data = pd.read_excel(i,index_col=0)
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    #finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    originlen = len(data)
    tempover = []
    smote = ADASYN(sampling_strategy=0.5)
    new_X, new_y = smote.fit_sample(finaldata,output)  # your imbalanced dataset is in X,y
    temp = pd.concat([new_X,new_y],axis=1)
    temp = temp.iloc[originlen:,:]
    sampleADASYN.append(temp)

In [ ]:
# 合併兩者 此法可能會少數據
allSample = []
temp = []
# 合併 Adsyn SMOTE
for i in range(len(sampleSMOTE)):
    #temp = randomAdsyn[i] + randomSMOTE[i]
    sampleADASYNpd = pd.DataFrame(sampleADASYN[i])
    sampleSMOTEpd = pd.DataFrame(sampleSMOTE[i])
    sampletemp = pd.concat([sampleADASYNpd,sampleSMOTEpd],axis=0)
    allSample.append(sampletemp)

In [ ]:
out = allSample[0].iloc[:,allSample[0].shape[1]-1]
outc = Counter(out)
print(outc)

In [ ]:
# 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
mergeRandom = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    data.Class= data.Class.str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    # 合併原先 data 跟 新生成的值
    mergeRandom = pd.concat([data,allSample[ii]],axis=0)

    output = mergeRandom.iloc[:,l];
    finaldata = mergeRandom.iloc[:,:l]
    c = Counter(output)
    print(c)
    clf = DecisionTreeClassifier()
    clf = clf.fit(finaldata, output) # 使用 decision tree 預測


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
    
    le = preprocessing.LabelEncoder() # 使 string 轉成

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)


